# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_data = pd.read_csv("output_data/cities.csv")
weather_data

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Río Gallegos,-51.62,-69.22,17.60,92,6,3.36,AR,1595294892
1,1,Punta Arenas,-53.15,-70.92,30.20,92,2,5.82,CL,1595294687
2,2,Rikitea,-23.12,-134.97,68.02,65,79,15.70,PF,1595294893
3,3,Coquimbo,-29.95,-71.34,53.60,87,90,1.48,CL,1595294893
4,4,Hermanus,-34.42,19.23,59.00,54,4,1.99,ZA,1595294893
...,...,...,...,...,...,...,...,...,...,...
568,568,Kachug,53.96,105.88,61.07,51,82,2.51,RU,1595294955
569,569,Goderich,43.75,-81.72,73.00,81,34,9.42,CA,1595294955
570,570,Hoi An,15.88,108.33,87.80,70,20,4.70,VN,1595294955
571,571,At-Bashi,41.17,75.81,55.44,62,8,1.79,KG,1595294955


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [11]:

fig = gmaps.figure(map_type='SATELLITE')
heatmap_locs = weather_data[["Lat","Lng"]]
humidity= weather_data["Humidity"].astype(float)
maxintensity = humidity.max()

heatmap_layer= gmaps.heatmap_layer(
    heatmap_locs,weights=humidity,max_intensity=maxintensity, point_radius =4,dissipating=False)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# A max temperature < 80 degrees but > 70.
# Wind speed < 10mph
# Cloudiness = 0
# drop any rows that don't contain any of the above


ideal_temp = weather_data.loc[
    (weather_data["Max Temp"]<80) & 
    (weather_data["Max Temp"]>70) & 
    (weather_data["Wind Speed"]<10) & 
    (weather_data["Cloudiness"]==0)]


# Dropping any null values
ideal_temp = ideal_temp.dropna(how='any')
ideal_temp.reset_index(inplace=True)
ideal_temp




,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,90,90,Oktyabr'skoye,43.06,44.74,75.20,57,0,8.95,RU,1595294902
1,109,109,Mazara del Vallo,37.66,12.59,71.60,88,0,3.36,IT,1595294905
2,127,127,Tieli,46.95,128.05,75.31,63,0,9.57,CN,1595294907
3,137,137,Loralai,30.37,68.60,79.03,45,0,2.84,PK,1595294908
4,142,142,Sestri Levante,44.27,9.40,71.01,88,0,4.72,IT,1595294908
5,173,173,Marsá Maţrūḩ,31.35,27.25,71.60,88,0,5.82,EG,1595294912
6,177,177,Zhanakorgan,43.91,67.25,78.89,20,0,7.14,KZ,1595294912
7,238,238,Boende,-0.22,20.87,70.11,75,0,1.79,CD,1595294918
8,260,260,Talaya,61.38,152.75,70.11,37,0,5.97,RU,1595294922
9,286,286,Caravelas,-17.71,-39.25,70.21,89,0,6.82,BR,1595294925


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
